In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

In [3]:
df_encoded = pd.read_csv('../Data/dataset.csv')
from sklearn.preprocessing import LabelEncoder
label_encoders = {}

for col in ["Area","Item"]:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

X = df_encoded.drop(columns=["hg/ha_yield"])
y = df_encoded["hg/ha_yield"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)